# Lab 3. Реалізація основних асиметричних криптосистем

## Мета
Реалізувати протокол розподілу симетричного ключа за допомогою
асиметричних криптосистем та порівняти ефективність різних підходів.

У даній роботі досліджуються два підходи:
1) передача ключа за допомогою RSA;
2) узгодження ключа за допомогою протоколу Діффі–Геллмана.


In [ ]:
import secrets
import time
from math import gcd

def is_even(n):
    return n % 2 == 0

def miller_rabin(n, k=10):
    if n < 4:
        return n in (2, 3)
    if is_even(n):
        return False

    d = n - 1
    s = 0
    while d % 2 == 0:
        d //= 2
        s += 1

    for _ in range(k):
        a = secrets.randbelow(n - 3) + 2
        x = pow(a, d, n)
        if x in (1, n - 1):
            continue
        for _ in range(s - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

def gen_rsa_prime(bits, k=10):
    while True:
        p = secrets.randbits(bits) | (1 << (bits-1)) | 1
        if miller_rabin(p, k):
            return p

def gen_rsa_keypair(bits=1024):
    p = gen_rsa_prime(bits//2)
    q = gen_rsa_prime(bits//2)
    n = p * q
    phi = (p-1)*(q-1)

    e = 65537
    if gcd(e, phi) != 1:
        raise RuntimeError("e not coprime")

    d = pow(e, -1, phi)
    return (e, n), (d, n)

def rsa_encrypt(m, pub):
    e, n = pub
    return pow(m, e, n)

def rsa_decrypt(c, priv):
    d, n = priv
    return pow(c, d, n)


In [ ]:
def rsa_key_transport(bits=1024):
    # Bob generates RSA keys
    pub, priv = gen_rsa_keypair(bits)

    # Alice generates symmetric key
    sym_key = secrets.randbits(256)

    # Alice encrypts key with Bob's public key
    t0 = time.perf_counter()
    c = rsa_encrypt(sym_key, pub)
    recovered = rsa_decrypt(c, priv)
    t1 = time.perf_counter()

    assert sym_key == recovered

    return {
        "bits": bits,
        "time_s": t1 - t0,
        "key_match": sym_key == recovered
    }

rsa_results = [rsa_key_transport(bits=b) for b in (1024, 2048)]
rsa_results


[{'bits': 1024, 'time_s': 0.004877523999994082, 'key_match': True},
 {'bits': 2048, 'time_s': 0.032065359000000626, 'key_match': True}]

## Етап 1. Протокол розподілу ключа на основі RSA (key transport)

У протоколі RSA Bob генерує пару ключів (публічний/приватний), а Alice генерує
випадковий симетричний ключ і **передає** його Bob у зашифрованому вигляді.

### Перевірка коректності
Коректність протоколу перевірялась умовою:
- розшифрований ключ Bob **дорівнює** ключу Alice (`key_match = True`).

У проведених експериментах для 1024 та 2048 біт умова виконалась.

### Швидкодія (експериментальні дані)
Час операцій шифрування+розшифрування симетричного ключа:

- **RSA 1024 біт**: 0.00488 с  
- **RSA 2048 біт**: 0.03207 с  

При збільшенні розрядності ключа RSA з 1024 до 2048 біт час виконання
зріс приблизно у **6–7 разів**, що пояснюється ускладненням операцій
модульної експонентації на числах більшої розрядності.

### Висновок по RSA
RSA-передача ключа забезпечує простий та наочний механізм розподілу
симетричного ключа, однак час операції істотно зростає зі збільшенням
розміру ключа.


## Етап 2. Протокол узгодження ключа на основі Діффі–Геллмана (DH)

У протоколі DH симетричний ключ **не передається** каналом:
Alice та Bob обмінюються публічними значеннями і незалежно обчислюють
однаковий спільний секрет.

Перевірка коректності:
- обчислений ключ Alice дорівнює ключу Bob.


In [ ]:
import secrets
import time
import pandas as pd

# We need a prime p and generator g.
# For simplicity and correctness in lab context:
# - generate a random prime p of required bit length (using miller_rabin)
# - choose small generator g = 2 (works for demo; not proving primitive root)

def gen_prime(bits, k=10):
    while True:
        n = secrets.randbits(bits) | (1 << (bits-1)) | 1
        if miller_rabin(n, k):
            return n

def dh_key_agreement(bits=1024):
    # public parameters
    p = gen_prime(bits)
    g = 2

    # private keys
    a = secrets.randbelow(p-2) + 2
    b = secrets.randbelow(p-2) + 2

    # public keys
    A = pow(g, a, p)
    B = pow(g, b, p)

    # shared secrets
    t0 = time.perf_counter()
    s1 = pow(B, a, p)
    s2 = pow(A, b, p)
    t1 = time.perf_counter()

    assert s1 == s2
    return {"bits": bits, "time_s": t1 - t0, "key_match": True}

dh_results = [dh_key_agreement(bits=b) for b in (1024, 2048)]
dh_results


[{'bits': 1024, 'time_s': 0.009560430999954406, 'key_match': True},
 {'bits': 2048, 'time_s': 0.06321275800007697, 'key_match': True}]

## Етап 2. Протокол узгодження ключа на основі Діффі–Геллмана (DH)

У протоколі Діффі–Геллмана симетричний ключ **не передається** напряму.
Alice та Bob обмінюються публічними значеннями та незалежно обчислюють
однаковий спільний секрет.

### Перевірка коректності
Коректність перевірялась умовою:
- обчислений ключ Alice дорівнює ключу Bob (`key_match = True`).

У проведених експериментах для 1024 та 2048 біт умова виконалась.

### Швидкодія (експериментальні дані)
Час обчислення спільного секрету (дві модульні експонентації):

- **DH 1024 біт**: 0.00956 с  
- **DH 2048 біт**: 0.06321 с  

При збільшенні розрядності з 1024 до 2048 біт час зріс приблизно у **6–7 разів**,
що узгоджується з ростом вартості модульної експонентації для більших чисел.

### Висновок по DH
DH забезпечує протокол узгодження ключа без передачі самого ключа каналом,
але вимагає модульних експонентацій, час яких суттєво зростає зі збільшенням
розрядності параметрів.


## Етап 3. Порівняння ефективності RSA та DH (за часом)

Було реалізовано один протокол розподілу ключа двома асиметричними підходами:
- **RSA (key transport)** — ключ генерується Alice і передається Bob у зашифрованому вигляді;
- **DH (key agreement)** — ключ не передається, а узгоджується обома сторонами.

### Порівняння часу виконання
Експериментальні результати:

- **1024 біт**
  - RSA: 0.00488 с
  - DH:  0.00956 с

- **2048 біт**
  - RSA: 0.03207 с
  - DH:  0.06321 с

У проведених вимірюваннях RSA виявився швидшим приблизно у **2 рази**
для обох розрядностей. Це пов’язано з тим, що в реалізації DH фактично
виконуються дві модульні експонентації для обчислення спільного секрету,
тоді як у RSA виконується шифрування та розшифрування одного короткого
симетричного ключа.

### Принципова різниця підходів
- RSA: ключ **передається** (але захищено асиметричним шифруванням).
- DH: ключ **не передається** — обчислюється незалежно обома сторонами.


## Висновки до лабораторної роботи №3

1. Реалізовано протокол розподілу симетричного ключа за допомогою двох
   різних асиметричних підходів: RSA (передача ключа) та DH (узгодження ключа).
2. Для обох протоколів виконано перевірку коректності: у всіх експериментах
   ключі збіглися (`key_match = True`).
3. Експериментально встановлено, що зі збільшенням розрядності з 1024 до 2048 біт
   час виконання зростає приблизно у 6–7 разів як для RSA, так і для DH,
   що пояснюється збільшенням вартості модульної експонентації.
4. За критерієм часу в проведених вимірюваннях RSA показав вищу швидкодію
   (приблизно у 2 рази швидше за DH для 1024 та 2048 біт).
5. Протоколи мають принципову різницю:
   RSA передає ключ у зашифрованому вигляді, а DH узгоджує ключ без передачі,
   що робить DH важливим для протоколів з вимогою “ключ не передавати каналом”.
